In [66]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 numbers=6,
                 extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.numbers = numbers
        self.extra = extra

    @property
    def backend(self):
        return self._backend

    @property
    def many(self):
        return self._many

    @staticmethod
    def pallottoliere(max_nums):
        while True:
            yield rnd.randint(1, max_nums)

    @staticmethod
    def choice(nums, max_nums):
        whole_numbers = list(range(1, max_nums+1))

        def extraction():
            single = rnd.choice(whole_numbers)
            whole_numbers.remove(single)

            return single

        return frozenset((extraction() for _ in range(nums)))

    @staticmethod
    def sample(nums, max_nums):
        whole_numbers = list(range(1, max_nums+1))

        return frozenset(rnd.sample(whole_numbers, k=nums))

    def randint(self, nums, max_nums):
        combo = set()
        while len(combo) < nums:
            combo.add(next(self.pallottoliere(max_nums)))

        return frozenset(combo)

    def manySamples(self, method):
        size = self._many or 1

        def sampler(backend):
            samples = [method(backend) for _ in range(size)]

            return rnd.choice(samples)

        return sampler

    def extract(self, backend):
        combo = backend(self.numbers, self.max_numbers)
        extra = backend(self.extra, self.max_extra) or None

        return combo, extra

    def __call__(self, backend='sample', many=1, verbose=True):
        self._backend = backend
        self._many = many

        backend = eval(backend,
                       {'__builtins__': {}},
                       {'choice': self.choice, 'randint': self.randint, 'sample': self.sample})

        combo, extra = self.manySamples(self.extract)(backend)

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        if verbose:
            print('Estrazione del:', now, '\nNumeri Estratti:', *sorted(combo))
            if extra is not None:
                print('Superstar:', *sorted(extra))  # type: ignore

        return combo, extra

In [67]:
superenalotto = Lotteria(max_numbers=90, max_extra=90, numbers=6, extra=0)

In [68]:
superenalotto(backend='randint', many=10_000);

Estrazione del: 11/06/2023 16:34 
Numeri Estratti: 26 43 44 56 61 89


In [69]:
eurojack = Lotteria(max_numbers=50, max_extra=12, numbers=5, extra=2)

In [70]:
eurojack(backend='sample', many=10_000);

Estrazione del: 11/06/2023 16:34 
Numeri Estratti: 1 9 15 17 45
Superstar: 6 8


In [71]:
eurojack(backend='choice', many=100_000);

Estrazione del: 11/06/2023 16:34 
Numeri Estratti: 4 17 30 32 49
Superstar: 4 5


In [72]:
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b, many=0)
#    print('backend:', eurojack.backend)
#
# print('\n################# many ####################\n')
#
# for b in ['choice', 'int', 'sample']:
#    print('######')
#    eurojack(backend=b)
#    print('backend:', eurojack.backend)

In [73]:
superenalotto.extra = 1
superenalotto();

Estrazione del: 11/06/2023 16:34 
Numeri Estratti: 12 14 21 66 74 77
Superstar: 13


In [74]:
superenalotto(backend='randint');

Estrazione del: 11/06/2023 16:34 
Numeri Estratti: 3 9 23 30 52 58
Superstar: 79


In [75]:
superenalotto.extra

1

In [76]:
superenalotto.many

1

In [77]:
winForLife = Lotteria(20, 20, 10, 1)
winForLife(backend='choice', many=100_000);

Estrazione del: 11/06/2023 16:34 
Numeri Estratti: 1 2 7 8 12 14 16 17 18 19
Superstar: 8


In [78]:
# trying to set backend via attribute: fails as expected
winForLife.backend = 'int'

AttributeError: property 'backend' of 'Lotteria' object has no setter